# How to open .npz with PYvista

This is a small script to open EUHFORIA .npz files with PyVista. 

Pysivsta is a 3D plotting and mesh analysis through a streamlined interface for the Visualization Toolkit (VTK). Its very well documented and has a lot of useful features that we can apply to make our plots nicer. The main PyVista page is: https://docs.pyvista.org/version/stable/

The main caveat that there is for visualizing EUHFORIA snapshots with PyVista  is that we need to convert the grid from the .npz files into a PyVista-readable format. This can be done by using a "new" io file in EUHFORIA, which adds this functionality. So there are two different ways we can proceed:

    1) We modify the io so that the load_heliospheric_data contains the property convert_to_pv_mesh

    2) We append the path to this new io, and use that io instead of EUHFORIA's included io. --> As I didn't want to modify EUHFORIA files, I appended the path and worked with this new io somewhere else.
    
NOTE: I don't import any EUHFORIA package so either path just relies on the io.
    



In [2]:
import numpy as np   #--> normal numpy
import pyvista as pv #--> you will need to install this package
#import vtk
#from vtk import *
#from vtk.util.numpy_support import vtk_to_numpy
import os,re, sys         #--> this is for prasing paths and filenames.
import matplotlib.pyplot as plt #--> normal matplotlib
###############
#THIS IS THE PATH APPEND THAT WILL TELL PYTHON WHERE TO LOOK FOR THE IO
sys.path.append('C:/Users/u0142106/Desktop/Barcelona_Project/shock_tracer-master/') #--> Modify this
###############
import shock_tracer.io_Tony as io #--> import the io from the new folder
import shock_tracer.constants as constants #--> these constants are optional, and are in the EUHFORIA folder as well.
#from astropy import units    

## This is how to load the data using the new io

The syntax is the same as the one in the EUHFORIA files. The new thing is that you need to include the scale factors (the numbers after each of the variables). This will scale the numbers back to SI units. If not they are in another units and the magnitudes will look a bit off.

In [ ]:
npz_file  = 'C:/Users/u0142106/Downloads/euhforia_sep232020T23h34m_ir_2020-09-23T23-33-33.npz' #--> Modify this
sw_data = io.load_heliospheric_data(npz_file,
                                    {"r":1,
                                     "lon":1, 
                                     "clt":1,
                                     "vr" : 1e3,
                                     "vlon" :1e3,
                                     "vclt":1e3,
                                     "Br":1e-9,
                                     "Blon":1e-9,
                                     "Bclt":1e-9,
                                     "n":1e6},
                                    r_unit=constants.au)

## This is the conversion to the PyVista mesh

This property is not in the EUHFORIA io, so if you try to load it from this file, it will complain. Make sure that you are pointing to the right file.

There is no need to convert r, lon and clt, as they are just used to convert into the proper grid by the io. Then each of the variables you will call it as they are named.

In [ ]:
sw_data.convert_to_pv_mesh(['vr','vlon','vclt','Br','Blon','Bclt','n'],delete=True)
sw_data.pvgrid
sw_data.pvgrid['vr'] = sw_data.pvgrid['vr']*1e-3  #--> to convert to km/s

In [ ]:
sw_data.pvgrid

## Now to the plotting part

It is pretty simple. The syntax is also intuitive.

First you set a plotter (pv.Plotter). The options that are inside the parenthesis correspond if you are using a Jupyter Notebook. As it is right now, it should prompt a new window with an interactive plotting screen, where you can zoom, rotate and move.

IF you try to move this script into the cluster, then you would need to put "off_screen = True", as this will prevent PyVista from trying to render. You would also need to take out the "notebook=False" option, as you will not be working on a notebook environment.

In [ ]:
p   = pv.Plotter(off_screen=False,notebook=False)
p.clear() 
p.add_mesh(sw_data.pvgrid, scalars = 'n')
p.show()

# This is a more complete plotting 

All the documentation for plotting is in the PyVista website. As I put above, its super complete and has a lot of examples that are useful.

### Slices:

In order to do slices, you need to give it the normal of the plane where you want the slice.

### Surface:

The surfaces do not work in a cell_data type, but in a point_data type. This switch is done simply by compute_cell_sizes and then running a cell_data_to_point_data conversion.

Then in order to compute the different isocontours, you put the limits of the isocountours that you want. In this case its from 200 to 800, 5 levels.

In order to do a slice at the inner boundary of EUHFORIA, you need to select the radius of the sphere and sample the data at that surface. This is done with the shell.sample, where you determine the resolution in $\theta$ and $\phi$, and you fix the radius at which you want the sample. In this case I put it at 0.11, but it can go lower. How much lower? I dont know.

### Plotting the slices and surfaces:

All is done by adding a mesh with p.add_mesh(data,**kwargs).

    - data: you put which data slice you want to plot.
    - "scalars": self explanatory. With this option you give the variable that you want to plot. 
    - "cmap" : same as matplotlib
    - "clim" : color bar limits
    - "scalar_bar_args ={'color':'black'}": This is for the colorbar legend to be in black. Depending on the background color you might need to change this.
    - "pv.Sphere": just puts a solid color sphere in the position you prescribe.

You can also fix text in space, and outside the the canvas where things are plotted.

    - p.add_point_labels([POSITION], ['STRING TO SHOW'], **kwargs)
            - [POSITION] : with respect to the pv grid.
            - ['STRING TO SHOW'] : just the label you want
            - **kwargs --> they are too many but also self explanatory. look through the documentation if in doubt
    
    - p.add_text('STRING TO SHOW', **kwargs)
            - 'STRING TO SHOW' : just what you want to show. I would suggest the time of the EUHFORIA slice.
            - **kwargs : same as above, just look into the documentation if in doubt.


In [ ]:
vr_slice_xy= sw_data.pvgrid.slice(normal=[0, 0, 1],progress_bar = True)
vr_slice_xz= sw_data.pvgrid.slice(normal=[0, 1, 0],progress_bar = True)
surf = sw_data.pvgrid.compute_cell_sizes(length=False, volume=False)
surf = surf.cell_data_to_point_data(progress_bar = True)
contours = surf.contour(np.linspace(200, 800, 5),progress_bar = True)
r = 0.11
shell = pv.Sphere(radius=r, theta_resolution=64, phi_resolution=64)
depth_slice = shell.sample(sw_data.pvgrid,progress_bar = True)
p   = pv.Plotter(off_screen=False,notebook=False)
p.clear() 
p.background_color = "white"
p.add_mesh(contours,scalars='vr',clim = [200,201],opacity = 0.5,cmap = 'hsv')
p.add_mesh(vr_slice_xy, scalars = 'vr',cmap = 'turbo',clim =[200,800],scalar_bar_args ={'color':'black'})
p.add_mesh(vr_slice_xz, scalars = 'vr',cmap = 'turbo',clim =[200,800],scalar_bar_args ={'color':'black'})
p.add_mesh(depth_slice, scalars = 'vr', cmap = 'turbo',clim =[200,800],scalar_bar_args ={'color':'black'})
p.add_mesh(pv.Sphere(radius=0.1, center=[0,0,0]), color='orange')
p.add_mesh(pv.Sphere(radius=0.025, center=[1,0,0]), color='green')
p.add_point_labels([[1,0.02,0.02]], ['Earth'], italic=False, font_size=20,show_points=False,
                        point_size=24,render_points_as_spheres=True,text_color='green',
                        shape=None,fill_shape=False, margin=60, always_visible=True)
p.add_point_labels([[0,0.1,0.2]], ['Sun'], italic=False, font_size=20,show_points=False,
                        point_size=24,render_points_as_spheres=True,text_color='orange',
                        shape=None,fill_shape=False, margin=60, always_visible=True)
p.add_text('String to show', position='upper_left',
                   color='black',shadow=True, font_size=18)
p.show()

In [ ]:
vr_slice_xy= sw_data.pvgrid.slice(normal=[0, 0, 1],progress_bar = True)
vr_slice_xz= sw_data.pvgrid.slice(normal=[0, 1, 0],progress_bar = True)
surf = sw_data.pvgrid.compute_cell_sizes(length=False, volume=False)
surf = surf.cell_data_to_point_data(progress_bar = True)
contours = surf.contour(np.linspace(200, 400, 2),progress_bar = True)
contours_1 = surf.contour(np.linspace(600, 700, 2),progress_bar = True)
r = 0.11
shell = pv.Sphere(radius=r, theta_resolution=64, phi_resolution=64)
depth_slice = shell.sample(sw_data.pvgrid,progress_bar = True)
p   = pv.Plotter(off_screen=False,notebook=False)
p.clear() 
p.background_color = "white"
p.add_mesh(contours,scalars='vr',clim = [200,701],opacity = 0.3,cmap = 'seismic')
p.add_mesh(contours_1,scalars='vr',clim = [200,701],opacity = 0.8,cmap = 'seismic')
#p.add_mesh(vr_slice_xy, scalars = 'vr',cmap = 'turbo',clim =[200,800],scalar_bar_args ={'color':'black'})
#p.add_mesh(vr_slice_xz, scalars = 'vr',cmap = 'turbo',clim =[200,800],scalar_bar_args ={'color':'black'})
#p.add_mesh(depth_slice, scalars = 'vr', cmap = 'turbo',clim =[200,800],scalar_bar_args ={'color':'black'})
p.add_mesh(pv.Sphere(radius=0.1, center=[0,0,0]), color='orange')
p.add_mesh(pv.Sphere(radius=0.025, center=[1,0,0]), color='green')
p.add_point_labels([[1,0.02,0.02]], ['Earth'], italic=False, font_size=20,show_points=False,
                        point_size=24,render_points_as_spheres=True,text_color='green',
                        shape=None,fill_shape=False, margin=60, always_visible=True)
p.add_point_labels([[0,0.1,0.2]], ['Sun'], italic=False, font_size=20,show_points=False,
                        point_size=24,render_points_as_spheres=True,text_color='orange',
                        shape=None,fill_shape=False, margin=60, always_visible=True)
p.add_text('String to show', position='upper_left',
                   color='black',shadow=True, font_size=18)
p.show()

# This is for including Field lines

Now that you can plot all the possible variables of EUFHORIA, lets try to see the field lines. This is a bit more complicated, as it needs multiple things to work:

1)It needs to know the B (or V) in a vector form. Right now each of the components is separated from the other, so it doesnt know that B (V) is actually a vector.

2) It needs to know where field lines should pass by.

3) How many and how long they have to be.

4) etc

We need this new function that comes from the shock tracer files as well.

In [3]:
import shock_tracer.transform as transform

As sw_data was already converted to a pv_grid and has now new "object" properties, we need to reload the snapshot. If the operations that we are going to do from this point onwards are done as soon as the file is loaded and before we convert it to a pv_grid, then all good.

In [ ]:
sw_data = io.load_heliospheric_data(npz_file,
                                    {"r":1,
                                     "lon":1, 
                                     "clt":1,
                                     "vr" : 1e3,
                                     "vlon" :1e3,
                                     "vclt":1e3,
                                     "Br":1e-9,
                                     "Blon":1e-9,
                                     "Bclt":1e-9,
                                     "n":1e6},
                                    r_unit=constants.au)

We need to generate a mesh with numpy such that the conversion is possible. This is basically a spherical coordinate system that comes from EUHFORIA

In [ ]:
r,t,p = np.meshgrid(sw_data.grid.center_coords.r,
                sw_data.grid.center_coords.clt, 
                sw_data.grid.center_coords.lon,
                indexing="ij" )

Now we do the transformation from a spherical vector to cartesian vector. This can also be done some other way, but this has worked for me so far, so:

In [ ]:
B = transform.spherical_vector_to_cartesian([sw_data.Br[1:,:,:],-sw_data.Bclt[:,1:,:],sw_data.Blon[:,:,1:]], [r,t,p])

Then we redo the conversion to the pyvista grid, and redo the vr.

The other line is generating the variable B, which is the magnetic field. Its one variable, but its a vector. It uses the values that we prescribed above, and generates a new "variable" in the sw_data

In [ ]:
sw_data.convert_to_pv_mesh(['vr','vlon','vclt','Br','Blon','Bclt','n','log10(nscaled)'],delete=True)
sw_data.pvgrid['vr'] = sw_data.pvgrid['vr']*1e-3  #--> to convert to km/s

sw_data.pvgrid.cell_data['B'] = np.column_stack ((sw_data.slice3D(B[0],sw_data.pv_idx_lims).T.ravel("C"),
                                                 sw_data.slice3D(B[1],sw_data.pv_idx_lims).T.ravel("C"),
                                                 sw_data.slice3D(B[2],sw_data.pv_idx_lims).T.ravel("C")))

We'll work now with point_data variables, so we need to convert all the cell_data variables to point_data. This is for the line tracing, as it cant work with the cell_data, it needs the point_data.

In [ ]:
mesh =sw_data.pvgrid.cell_data_to_point_data()
mesh['b'] = mesh['B']*1e-9

# Now we start the fieldline tracing

These next code blocks will do the basics for fieldline tracing

## Surfaces

It is important also to know which are the surfaces that we want the field lines to cross. So here is a short intro to different types of surfaces that can be useful.

The idea is the same with all. You "select" a type of surface that you want to plot, and give PyVista the resolution at which you want plot. For example:

-   If you want a smooth sphere you want more than 10 points in the phi and in the theta.
    - sphere = pv.Sphere(radius=1.5, phi_resolution=15, theta_resolution=15)
    - this gives a sphere of radius 1.5, with 15 points in the phi (latitude) and 15 points in the theta (longitude)
-   If you want a Torus, then you need to give the 
    - torus  = pv.ParametricTorus(ringradius=1.0, crosssectionradius=0.5,u_res=10, v_res=10, w_res=10)
    - this gives a torus of inner radius = 1, with a crosssection of 0.5, with a resolution in x, y and z of 10 points each.
-   If you will want that field lines traced are only the ones that are going towards Earth, and not to the back of the Sun
    -part_ellipsoid = pv.ParametricEllipsoid(-1, 1, 1,u_res=10, v_res=10, w_res=10,max_u =np.radians(300),min_u=np.radians(90),max_v = np.radians(105),min_v=np.radians(75))
    
    -this gives this section of an ellipsoid. Try to plot it yourself!
    
    
## Resolutions are important, as they will give you the amount of lines that will be traced through the surface.



In [ ]:
sphere = pv.Sphere(radius=1.5, phi_resolution=15, theta_resolution=15)
torus  = pv.ParametricTorus(u_res=10, v_res=10, w_res=10)
part_ellipsoid = pv.ParametricEllipsoid(-1, 1, 1,u_res=10, v_res=10, w_res=10,
                                          max_u = np.radians(300),min_u=np.radians(90), 
                                          max_v = np.radians(105),min_v=np.radians(75))

## Now the tracer 

sc_imf_source0 is the variable that has the information of the fieldlines.

- you give the surface which the lines need to go through --> any of the parametric surfaces that we had before
    - in this case its the part_ellipsoid
- you need to give which is the vector that you want to plot
    - in this case is 'b' (careful, that it is not 'B', because we redefined it to have the proper units)
- terminal_speed and compute_vorticity
    - for the aplication that we need now, they only make the calculations MUUUUCH slower. 
- progress_bar is just for knowing that the script is working.





In [ ]:
sc_imf_sourceEllipsoid = mesh.streamlines_from_source(part_ellipsoid,'b',
                                                      terminal_speed=1e-22,compute_vorticity=False,progress_bar = True) 
sc_imf_torus = mesh.streamlines_from_source(torus,'b',
                                                      terminal_speed=1e-22,compute_vorticity=False,progress_bar = True) 
sc_imf_sphere = mesh.streamlines_from_source(sphere,'b',
                                                      terminal_speed=1e-22,compute_vorticity=False,progress_bar = True) 

In [ ]:
p   = pv.Plotter(off_screen=False,notebook=False)
p.clear() 
p.background_color = "white"
p.add_mesh(part_ellipsoid,color = 'red')
p.add_mesh(torus,color = 'blue',opacity = 0.3)
p.add_mesh(sphere,color = 'green',opacity = 0.3)
p.add_mesh(sc_imf_sourceEllipsoid.tube(radius=0.001),color = 'red')
p.add_mesh(sc_imf_torus.tube(radius=0.003),color = 'blue')
p.add_mesh(sc_imf_sphere.tube(radius=0.002),color = 'green')
p.show()

## Play around with the resolution and with the linewidths to see how they affect the plotting.

For my particular case I found that tubes of radius 0.001 were OK. And as I only use spacecraft at Earth and close positions, the spheres that I use are of 1*1 resolution, and of radius 0.025. These might be too big if you want more of a mesh of spacecraft close to Earth. Play around with what suits your problem the most!

### Personally if the case is that you want the field lines that go through this "mesh" of spacecraft close to Earth, I would do as the Ellipsoid surface, but moved to 1au.


In [ ]:
Earth_sphere   = pv.Sphere(radius=0.025, center=[1,0,0],theta_resolution=1, phi_resolution=1)
Mid_sphere   = pv.Sphere(radius=0.025, center=[0.7,0,0],theta_resolution=1, phi_resolution=1)
Close_sphere   = pv.Sphere(radius=0.025, center=[0.3,0,0],theta_resolution=1, phi_resolution=1)

sc_imf_sourceEarth = mesh.streamlines_from_source(Earth_sphere,'b',terminal_speed=1e-22,compute_vorticity=False,progress_bar = True) 
sc_imf_sourceClose = mesh.streamlines_from_source(Close_sphere,'b',terminal_speed=1e-22,compute_vorticity=False,progress_bar = True) 
sc_imf_sourceMid = mesh.streamlines_from_source(Mid_sphere,'b',terminal_speed=1e-22,compute_vorticity=False,progress_bar = True) 

Earth_30_sphere   = pv.Sphere(radius=0.025, center=[np.cos(np.radians(30))*1,np.sin(np.radians(30))*1,0],theta_resolution=1, phi_resolution=1)
Close_30_sphere   = pv.Sphere(radius=0.025, center=[0.3*np.cos(np.radians(30)),0.3*np.sin(np.radians(30)),0],theta_resolution=1, phi_resolution=1)
Mid_30_sphere   = pv.Sphere(radius=0.025, center=[0.7*np.cos(np.radians(30)),0.7*np.sin(np.radians(30)),0],theta_resolution=1, phi_resolution=1)
sc_imf_sourceEarth_30 = mesh.streamlines_from_source(Earth_30_sphere,'b',terminal_speed=1e-22,compute_vorticity=False,progress_bar = True) 
sc_imf_sourceClose_30 = mesh.streamlines_from_source(Close_30_sphere,'b',terminal_speed=1e-22,compute_vorticity=False,progress_bar = True) 
sc_imf_sourceMid_30 = mesh.streamlines_from_source(Mid_30_sphere,'b',terminal_speed=1e-22,compute_vorticity=False,progress_bar = True) 

Earth__30_sphere   = pv.Sphere(radius=0.025, center=[np.cos(np.radians(-30))*1,np.sin(np.radians(-30))*1,0],theta_resolution=1, phi_resolution=1)
Close__30_sphere   = pv.Sphere(radius=0.025, center=[0.3*np.cos(np.radians(-30)),0.3*np.sin(np.radians(-30)),0],theta_resolution=1, phi_resolution=1)
Mid__30_sphere   = pv.Sphere(radius=0.025, center=[0.7*np.cos(np.radians(-30)),0.7*np.sin(np.radians(-30)),0],theta_resolution=1, phi_resolution=1)
sc_imf_sourceEarth__30 = mesh.streamlines_from_source(Earth__30_sphere,'b',terminal_speed=1e-22,compute_vorticity=False,progress_bar = True) 
sc_imf_sourceClose__30 = mesh.streamlines_from_source(Close__30_sphere,'b',terminal_speed=1e-22,compute_vorticity=False,progress_bar = True) 
sc_imf_sourceMid__30 = mesh.streamlines_from_source(Mid__30_sphere,'b',terminal_speed=1e-22,compute_vorticity=False,progress_bar = True) 

Earth_60_sphere   = pv.Sphere(radius=0.025, center=[np.cos(np.radians(60))*1,np.sin(np.radians(60))*1,0],theta_resolution=1, phi_resolution=1)
Close_60_sphere   = pv.Sphere(radius=0.025, center=[0.3*np.cos(np.radians(60)),0.3*np.sin(np.radians(60)),0],theta_resolution=1, phi_resolution=1)
Mid_60_sphere   = pv.Sphere(radius=0.025, center=[0.7*np.cos(np.radians(60)),0.7*np.sin(np.radians(60)),0],theta_resolution=1, phi_resolution=1)
sc_imf_sourceEarth_60 = mesh.streamlines_from_source(Earth_60_sphere,'b',terminal_speed=1e-22,compute_vorticity=False,progress_bar = True) 
sc_imf_sourceClose_60 = mesh.streamlines_from_source(Close_60_sphere,'b',terminal_speed=1e-22,compute_vorticity=False,progress_bar = True) 
sc_imf_sourceMid_60 = mesh.streamlines_from_source(Mid_60_sphere,'b',terminal_speed=1e-22,compute_vorticity=False,progress_bar = True) 

Earth__60_sphere   = pv.Sphere(radius=0.025, center=[np.cos(np.radians(-60))*1,np.sin(np.radians(-60))*1,0],theta_resolution=1, phi_resolution=1)
Close__60_sphere   = pv.Sphere(radius=0.025, center=[0.3*np.cos(np.radians(-60)),0.3*np.sin(np.radians(-60)),0],theta_resolution=1, phi_resolution=1)
Mid__60_sphere   = pv.Sphere(radius=0.025, center=[0.7*np.cos(np.radians(-60)),0.7*np.sin(np.radians(-60)),0],theta_resolution=1, phi_resolution=1)
sc_imf_sourceEarth__60 = mesh.streamlines_from_source(Earth__60_sphere,'b',terminal_speed=1e-22,compute_vorticity=False,progress_bar = True) 
sc_imf_sourceClose__60 = mesh.streamlines_from_source(Close__60_sphere,'b',terminal_speed=1e-22,compute_vorticity=False,progress_bar = True) 
sc_imf_sourceMid__60 = mesh.streamlines_from_source(Mid__60_sphere,'b',terminal_speed=1e-22,compute_vorticity=False,progress_bar = True) 


In [ ]:
cmap = 'hsv'
single_slice = sw_data.pvgrid.slice(normal=[0, 0, 1],progress_bar = True)

pl = pv.Plotter(off_screen = False,notebook = False)
#pl.add_mesh(sw_data.pvgrid,scalars='Br',cmap=cmap,clim=[-1,1])
#pl.view_xy()

pl.camera.position    = (2,0, 2)
pl.camera.focal_point = (0.5,0, 0.)

#pl.add_mesh(sphere,opacity = 0.2)
pl.add_mesh(pv.Sphere(radius=0.1, center=[0,0,0]), color='orange')
#Sun-Earth line
pl.add_mesh(pv.Sphere(radius=0.025, center=[1,0,0]), color='green')
pl.add_mesh(pv.Sphere(radius=0.025, center=[0.3,0,0]), color='blue')
pl.add_mesh(pv.Sphere(radius=0.025, center=[0.7,0,0]), color='red')
#+30
pl.add_mesh(pv.Sphere(radius=0.025, center=[np.cos(np.radians(30))*1,np.sin(np.radians(30))*1,0]), color='green')
pl.add_mesh(pv.Sphere(radius=0.025, center=[0.3*np.cos(np.radians(30)),0.3*np.sin(np.radians(30)),0]), color='blue')
pl.add_mesh(pv.Sphere(radius=0.025, center=[0.7*np.cos(np.radians(30)),0.7*np.sin(np.radians(30)),0]), color='red')
#-30
pl.add_mesh(pv.Sphere(radius=0.025, center=[np.cos(np.radians(-30))*1,np.sin(np.radians(-30))*1,0]), color='green')
pl.add_mesh(pv.Sphere(radius=0.025, center=[0.3*np.cos(np.radians(-30)),0.3*np.sin(np.radians(-30)),0]), color='blue')
pl.add_mesh(pv.Sphere(radius=0.025, center=[0.7*np.cos(np.radians(-30)),0.7*np.sin(np.radians(-30)),0]), color='red')
#+60
pl.add_mesh(pv.Sphere(radius=0.025, center=[np.cos(np.radians(60))*1,np.sin(np.radians(60))*1,0]), color='green')
pl.add_mesh(pv.Sphere(radius=0.025, center=[0.3*np.cos(np.radians(60)),0.3*np.sin(np.radians(60)),0]), color='blue')
pl.add_mesh(pv.Sphere(radius=0.025, center=[0.7*np.cos(np.radians(60)),0.7*np.sin(np.radians(60)),0]), color='red')
#-60
pl.add_mesh(pv.Sphere(radius=0.025, center=[np.cos(np.radians(-60))*1,np.sin(np.radians(-60))*1,0]), color='green')
pl.add_mesh(pv.Sphere(radius=0.025, center=[0.3*np.cos(np.radians(-60)),0.3*np.sin(np.radians(-60)),0]), color='blue')
pl.add_mesh(pv.Sphere(radius=0.025, center=[0.7*np.cos(np.radians(-60)),0.7*np.sin(np.radians(-60)),0]), color='red')
#pl.add_mesh(part_ellipsoid,color='lightblue')
#pl.add_mesh(
#    sc_imf_source0.tube(radius=0.001),
#    cmap='hsv',
#    ambient=0.5,
#)
pl.add_mesh(sc_imf_sourceEarth.tube(radius=0.001),color = 'green', ambient=0.5)
pl.add_mesh(sc_imf_sourceClose.tube(radius=0.001),color = 'blue',ambient=0.5)
pl.add_mesh(sc_imf_sourceMid.tube(radius=0.001),color = 'red',ambient=0.5)

pl.add_mesh(sc_imf_sourceEarth_30.tube(radius=0.001),color = 'green', ambient=0.5)
pl.add_mesh(sc_imf_sourceClose_30.tube(radius=0.001),color = 'blue',ambient=0.5)
pl.add_mesh(sc_imf_sourceMid_30.tube(radius=0.001),color = 'red',ambient=0.5)

pl.add_mesh(sc_imf_sourceEarth__30.tube(radius=0.001),color = 'green', ambient=0.5)
pl.add_mesh(sc_imf_sourceClose__30.tube(radius=0.001),color = 'blue',ambient=0.5)
pl.add_mesh(sc_imf_sourceMid__30.tube(radius=0.001),color = 'red',ambient=0.5)

pl.add_mesh(sc_imf_sourceEarth_60.tube(radius=0.001),color = 'green', ambient=0.5)
pl.add_mesh(sc_imf_sourceClose_60.tube(radius=0.001),color = 'blue',ambient=0.5)
pl.add_mesh(sc_imf_sourceMid_60.tube(radius=0.001),color = 'red',ambient=0.5)

pl.add_mesh(sc_imf_sourceEarth__60.tube(radius=0.001),color = 'green', ambient=0.5)
pl.add_mesh(sc_imf_sourceClose__60.tube(radius=0.001),color = 'blue',ambient=0.5)
pl.add_mesh(sc_imf_sourceMid__60.tube(radius=0.001),color = 'red',ambient=0.5)
#pl.add_mesh(single_slice,scalars='log10(Btot)' ,cmap=cmap,clim=[-10,-6])

surf = sw_data.pvgrid.compute_cell_sizes(length=False, volume=False)
surf = surf.cell_data_to_point_data(progress_bar = True)
contours = surf.contour(np.linspace(200, 400, 2),progress_bar = True)
contours_1 = surf.contour(np.linspace(600, 700, 2),progress_bar = True)
r = 0.11
shell = pv.Sphere(radius=r, theta_resolution=64, phi_resolution=64)
depth_slice = shell.sample(sw_data.pvgrid,progress_bar = True)

pl.add_mesh(contours,scalars='vr',clim = [200,701],opacity = 0.3,cmap = 'seismic')
pl.add_mesh(contours_1,scalars='vr',clim = [200,701],opacity = 0.8,cmap = 'seismic')
#pl.add_mesh(vr_slice_xy, scalars = 'vr',cmap = 'turbo',clim =[200,800],scalar_bar_args ={'color':'black'})
#pl.add_mesh(vr_slice_xz, scalars = 'vr',cmap = 'turbo',clim =[200,800],scalar_bar_args ={'color':'black'})
#p.add_mesh(depth_slice, scalars = 'vr', cmap = 'turbo',clim =[200,800],scalar_bar_args ={'color':'black'})

pl.add_point_labels([[1,0.02,0.02]], ['Earth'], italic=False, font_size=20,show_points=False,
                        point_size=24,render_points_as_spheres=True,text_color='green',
                        shape=None,fill_shape=False, margin=60, always_visible=True)
pl.add_point_labels([[0,0.1,0.2]], ['Sun'], italic=False, font_size=20,show_points=False,
                        point_size=24,render_points_as_spheres=True,text_color='orange',
                        shape=None,fill_shape=False, margin=60, always_visible=True)
pl.add_text('String to show', position='upper_left',
                   color='black',shadow=True, font_size=18)
pl.show()